<a href="https://colab.research.google.com/github/sincanmaulanaa/dicoding-machine-learning-development-exercises/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install google-play-scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.7 MB/s eta 0:00:00


In [3]:
# Mengimpor pustaka google_play_scraper untuk mengakses ulasan dan informasi aplikasi dari Google Play Store.
from google_play_scraper import app, reviews, Sort, reviews_all
import pandas as pd # Pandas untuk manipulasi dan analisis data
pd.options.mode.chained_assignment = None # Menonaktifkan peringatan chaining

import numpy as np # NumPy untuk komputasi numerik
seed = 0
np.random.seed(seed) # Mengatur seed untuk reproduktibilitas

import matplotlib.pyplot as plt #Matplotlib untuk visualisasi data
import seaborn as sns # Seaborn untuk visualisasi data statistik, mengatur gaya visualisasi

import datetime as dt # Manipulasi waktu dan tanggal
import re # Modul untuk bekerja dengan ekspresi reguler
import string # Berisi konstanta string, seperti tanda baca

from nltk.tokenize import word_tokenize # Tokenisasi teks
from nltk.corpus import stopwords # Daftar kata-kata berhenti dalam teks

In [4]:
!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory # Stemming (penghilangan imbuhan kata) dalam bahasa Indonesia
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory # Menghapus kata-kata berhenti dalam bahasa Indonesia

from wordcloud import WordCloud # Membuat visualisasi berbentuk awan kata (word cloud) dari teks

import nltk # Import pustaka NLTK (Natural Language Toolkit)
nltk.download('punkt') # Mengunduh dataset yang diperlukan untuk tokenisasi teks.
nltk.download('stopwords') # Mengunduh dataset yang berisi daftar kata-kata berhenti (stopwords) dalam berbagai bahasa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.5 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [74]:
# Mengambil semua ulasan dari aplikasi dengan ID 'com.byu.id' di Google Play Store.
# Proses scraping mungkin memerlukan beberapa saat tergantung pada jumlah ulasan yang ada.
scrapereview = reviews_all(
    'com.mobile.legends', # ID Aplikasi
    lang='id', # bahasa ulasan, default 'en'
    country='id', # negara, default 'us'
    sort=Sort.MOST_RELEVANT, # urutan ulasan, default Sort.MOST_RELEVANT
    count=10000 # jumlah maksimum ulasan yang ingin diambil
)

In [75]:
# Menyimpan ulasan dalam file CSV
import csv

with open('ulasan_mlbb.csv', mode='w', newline='', encoding='utf-8') as file:
  writer = csv.writer(file)
  writer.writerow(['Review']) # menulis header kolom
  for review in scrapereview:
    writer.writerow([review['content']]) # menulis konten ulasan ke dalam file CSV

In [76]:
# meload dataset
app_reviews_df = pd.DataFrame(scrapereview)
app_reviews_df.shape
app_reviews_df.head()
app_reviews_df.to_csv('ulasan_mlbb.csv', index=False)

# membuat dataframe dari hasil scrapereview
app_reviews_df = pd.DataFrame(scrapereview)

# menghitung jumlah baris dan kolom dalam dataframe
jumlah_ulasan, jumlah_kolom = app_reviews_df.shape
print('JUMLAH ULASAN: ', jumlah_ulasan)
print('JUMLAH KOLOM: ', jumlah_kolom)

JUMLAH ULASAN:  162000
JUMLAH KOLOM:  11


In [77]:
app_reviews_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,9495c1df-df88-4dd7-bf1d-888897aaa813,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Udh lama main mlbb, game 5v5 paling bagus sih ...",2,961,1.9.11.9836,2024-08-28 11:36:52,None,NaT,1.9.11.9836
1,aeec5abc-25b0-47a8-97de-cd43cd96f0e8,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Pembuatan system yang buruk kenapa saya selalu...,1,7313,1.9.11.9836,2024-09-03 13:00:05,None,NaT,1.9.11.9836
2,fbf63997-17e3-4ca9-a6af-7dda90055a02,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Gamenya bagus, udah gak kikir kayak dulu, tapi...",2,620,1.9.11.9836,2024-08-29 11:30:29,None,NaT,1.9.11.9836
3,eacf81fe-2126-44d7-bd0b-591e464eca91,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Bug yang terus berkelanjutan sering mengalami ...,2,949,1.9.11.9836,2024-08-24 13:05:22,None,NaT,1.9.11.9836
4,693ba8fc-cc98-421a-a62c-164f7f55ff3d,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"bagus sih ini, udh banyak event skin gratis ta...",1,1005,1.9.11.9836,2024-08-30 01:28:49,None,NaT,1.9.11.9836


In [78]:
app_reviews_df.info()

# terdapat kolom dengan nilai null, yaitu kolom
# reviewCreatedVersion, replyContent, repliedAt, appVersion

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162000 entries, 0 to 161999
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   reviewId              162000 non-null  object        
 1   userName              162000 non-null  object        
 2   userImage             162000 non-null  object        
 3   content               162000 non-null  object        
 4   score                 162000 non-null  int64         
 5   thumbsUpCount         162000 non-null  int64         
 6   reviewCreatedVersion  138846 non-null  object        
 7   at                    162000 non-null  datetime64[ns]
 8   replyContent          5809 non-null    object        
 9   repliedAt             5809 non-null    datetime64[ns]
 10  appVersion            138846 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 13.6+ MB


In [79]:
# Membuat DataFrame baru (clean_df) dengan menghapus baris
# yang memiliki nilai yang hilang (NaN) dari app_reviews_df
clean_df = app_reviews_df.dropna()

In [80]:
# menghapus baris duplikat dari dataframe clean_df
clean_df = clean_df.drop_duplicates()

In [81]:
# Menghitung jumlah baris dan kolom dalam DataFrame clean_df setelah menghapus duplikat
jumlah_ulasan_setelah_hapus_duplikat, jumlah_kolom_setelah_hapus_duplikat = clean_df.shape
print("JUMLAH ULASAN: ", jumlah_ulasan_setelah_hapus_duplikat)
print("JUMLAH KOLOM: ", jumlah_kolom_setelah_hapus_duplikat)

JUMLAH ULASAN:  4902
JUMLAH KOLOM:  11


In [82]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4902 entries, 21 to 161997
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              4902 non-null   object        
 1   userName              4902 non-null   object        
 2   userImage             4902 non-null   object        
 3   content               4902 non-null   object        
 4   score                 4902 non-null   int64         
 5   thumbsUpCount         4902 non-null   int64         
 6   reviewCreatedVersion  4902 non-null   object        
 7   at                    4902 non-null   datetime64[ns]
 8   replyContent          4902 non-null   object        
 9   repliedAt             4902 non-null   datetime64[ns]
 10  appVersion            4902 non-null   object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 459.6+ KB


In [83]:
# pre-processing text
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer import StemmerFactory

# Membersihkan teks dengan menghapus mention, hashtag, RT (retweet), tautan (link),
# angka, dan tanda baca. Selain itu, karakter newline diganti dengan spasi dan spasi
# ekstra pada awal dan akhir teks dihapus.

def cleaningText(text):
  text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
  text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
  text = re.sub(r'RT[\s]', '', text) # menghapus RT
  text = re.sub(r"http\S+", '', text) # menghapus link
  text = re.sub(r'[0-9]+', '', text) # menghapus angka
  text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka

  text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
  text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
  text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
  return text

def casefoldingText(text):
  text = text.lower() # mengubah teks menjadi huruf kecil
  return text

def tokenizingText(text):
  text = word_tokenize(text) # memecah atau membagi string teks menjadi token
  return text

def filteringText(text): # menghapus stopwords dalam teks
  listStopWords = set(stopwords.words('indonesian'))
  listStopWords1 = set(stopwords.words('english'))

  listStopWords.update(listStopWords1)
  listStopWords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])

  filtered = []
  for txt in text:
    if txt not in listStopWords:
      filtered.append(txt)

  text = filtered
  return text

def stemmingText(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
  # membuat object stemmer
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()

  # memecah teks menjadi daftar kata
  words = text.split()

  # menerapkan stemming pada setiap kata dalam daftar
  stemmed_words = [stemmer.stem(word) for word in words]

  # menggabungkan kata-kata yang telah distem
  stemmed_text = ' '.join(stemmed_words)
  return stemmed_text

def toSentence(list_words):
  sentence = ' '.join(word for word in list_words)
  return sentence


In [128]:
# menghapus kumpulan slang words atau kata-kata informal
slangwords = {
    "@": "di",
    "abis": "habis",
    "masi": "masih",
    "bgt": "banget",
    "maks": "maksimal",
    "yg": "yang",
    "trus": "terus",
    "udh": "sudah",
    "afk": "keluar",
    "tpi": "tapi",
    "trs": "terus",
    "tuh": "itu",
    "tp": "tapi",
    "dgn": "dengan",
    "gw": "saya",
    "gue": "saya",
    "kamu": "kamu",
    "km": "kamu",
    "dr": "dari",
    "gak": "tidak",
    "ga": "tidak",
    "nggak": "tidak",
    "blm": "belum",
    "sbnr": "sebenarnya",
    "mkn": "mungkin",
    "ngerti": "mengerti",
    "pd": "pada",
    "dmn": "dimana",
    "aja": "saja",
    "jd": "jadi",
    "kl": "kalau",
    "nih": "ini",
    "lg": "lagi",
    "krn": "karena",
    "ok": "oke",
    "skrg": "sekarang",
    "sy": "saya",
    "dlm": "dalam",
    "klian": "kalian",
    "bs": "bisa",
    "plg": "paling",
    "gaada": "tidak ada",
    "drpd": "daripada",
    "plis": "tolong",
    "bro": "teman",
    "sis": "saudara",
    "an": "anak",
    "swt": "kesal",
    "sori": "maaf",
    "blg": "bilang",
    "ntar": "nanti",
    "mlm": "malam",
    "siang": "siang",
    # Slang khusus Mobile Legends dan game lainnya
    "ml": "mobile legends",
    "noob": "pemain pemula",
    "gg": "good game",
    "mvp": "most valuable player",
    "rank": "peringkat",
    "gank": "serangan bersama",
    "buff": "peningkatan kekuatan",
    "nerf": "pengurangan kekuatan",
    "ulti": "ultimate skill",
    "troll": "pemain yang mengganggu",
    "spam": "mengulang-ulang",
    "lag": "keterlambatan koneksi",
    "fps": "frame per second",
    "skin": "penampilan karakter",
    "meta": "strategi terbaik",
    "op": "overpowered",
    "push": "mendongkrak peringkat",
    "afk": "away from keyboard",
    "ban": "dilarang",
    "winrate": "persentase kemenangan",
    "team": "tim",
    "solo": "bermain sendiri",
    "matchmaking": "pencocokan permainan",
    "report": "melaporkan",
    "toxic": "perilaku buruk",
    "feeder": "pemain yang sering mati",
    "jungler": "pemain di hutan",
    "mid": "jalur tengah",
    "bot": "jalur bawah",
    "top": "jalur atas",
    "mm": "marksman",
    "tank": "pemain bertahan",
    "support": "pemain pendukung",
    "mage": "penyihir",
    "assassin": "pembunuh",
    "dc": "putus koneksi",
    "yah": "ya"
}


def fix_slangwords(text):
  words = text.split()
  fixed_words = []

  for word in words:
    if word.lower() in slangwords:
      fixed_words.append(slangwords[word.lower()])
    else:
      fixed_words.append(word)

  fixed_text = ' '.join(fixed_words)
  return fixed_text

In [129]:
# mengimplementasikan semua step pre-processing text

# Membersihkan teks dan menyimpannya di kolom 'text_clean'
clean_df['text_clean'] = clean_df['content'].apply(cleaningText)

# Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di 'text_casefoldingText'
clean_df['text_casefoldingText'] = clean_df['text_clean'].apply(casefoldingText)

# Mengganti kata-kata slang dengan kata-kata standar dan menyimpannya di 'text_slangwords'
clean_df['text_slangwords'] = clean_df['text_casefoldingText'].apply(fix_slangwords)

# Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_tokenizingText'
clean_df['text_tokenizingText'] = clean_df['text_slangwords'].apply(tokenizingText)

# Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
clean_df['text_stopword'] = clean_df['text_tokenizingText'].apply(filteringText)

# Menggabungkan token-token menjadi kalimat dan menyimpannya di 'text_akhir'
clean_df['text_akhir'] = clean_df['text_stopword'].apply(toSentence)

In [130]:
clean_df

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir,polarity_score,polarity
21,ba103f06-17b8-4a79-af7f-65d00def2a42,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Sebenernya Game Nya biasa aja Cuman ya cuman, ...",2,98,1.9.11.9836,2024-09-05 12:12:11,"Dear Hero,\nTerima kasih atas tanggapan Anda, ...",2024-04-21 06:39:38,1.9.11.9836,Sebenernya Game Nya biasa aja Cuman ya cuman m...,sebenernya game nya biasa aja cuman ya cuman m...,sebenernya game nya biasa saja cuman ya cuman ...,"[sebenernya, game, nya, biasa, saja, cuman, ya...","[sebenernya, game, cuman, cuman, mines, ingame...",sebenernya game cuman cuman mines ingame ping ...,12,positive
67,3094953a-5aaf-4de6-b46a-50ce6b33fa5d,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Game nya lumayan seru sih tapi kenapa ya aku k...,1,170,1.9.11.9836,2024-08-30 13:41:03,"Dear Hero, Kami bertekad untuk menciptakan lin...",2024-07-17 16:37:27,1.9.11.9836,Game nya lumayan seru sih tapi kenapa ya aku k...,game nya lumayan seru sih tapi kenapa ya aku k...,game nya lumayan seru sih tapi kenapa ya aku k...,"[game, nya, lumayan, seru, sih, tapi, kenapa, ...","[game, lumayan, seru, kalo, main, suka, ngeleg...",game lumayan seru kalo main suka ngeleg sinyal...,-12,negative
80,9dbb0fbd-61f9-4728-9442-2a9238bbd4ca,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"setelah update, loading game lama sekali diawa...",1,49,1.9.8.9833,2024-09-03 09:23:17,"Dear Hero,\nMaaf atas ketidaknyamanan ini, sil...",2024-09-03 10:09:51,1.9.8.9833,setelah update loading game lama sekali diawal...,setelah update loading game lama sekali diawal...,setelah update loading game lama sekali diawal...,"[setelah, update, loading, game, lama, sekali,...","[update, loading, game, diawali, blackscreen, ...",update loading game diawali blackscreen menit ...,-2,negative
86,f405c452-0ef0-4838-b81f-d891beb3d569,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Dear moonton, event dan skin gamemu ini memang...",2,51,1.9.9.9834,2024-09-01 16:16:59,"Dear Hero, Kami bertekad untuk menciptakan lin...",2024-08-14 06:27:54,1.9.9.9834,Dear moonton event dan skin gamemu ini memang ...,dear moonton event dan skin gamemu ini memang ...,dear moonton event dan penampilan karakter gam...,"[dear, moonton, event, dan, penampilan, karakt...","[dear, moonton, event, penampilan, karakter, g...",dear moonton event penampilan karakter gamemu ...,-20,negative
87,3500f38c-b481-4f2c-b4db-49f56ba577b9,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Mohon developer match making nya di perbaiki, ...",3,7,1.8.93.9702,2024-08-29 05:33:31,"Dear Hero,\nKami bertekad untuk menciptakan li...",2024-08-29 06:52:00,1.8.93.9702,Mohon developer match making nya di perbaiki t...,mohon developer match making nya di perbaiki t...,mohon developer match making nya di perbaiki t...,"[mohon, developer, match, making, nya, di, per...","[mohon, developer, match, making, perbaiki, ti...",mohon developer match making perbaiki tim bene...,-12,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161989,0b847475-1f74-4544-b055-3719298a10a3,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Kualitas player yg bermain buruk, jelek, lemah...",1,0,1.5.88.6441,2024-02-04 03:40:39,"Dear Hero, Kami bertekad untuk menciptakan lin...",2024-02-07 03:19:32,1.5.88.6441,Kualitas player yg bermain buruk jelek lemah m...,kualitas player yg bermain buruk jelek lemah m...,kualitas player yang bermain buruk jelek lemah...,"[kualitas, player, yang, bermain, buruk, jelek...","[kualitas, player, bermain, buruk, jelek, lema...",kualitas player bermain buruk jelek lemah sand...,-17,negative
161990,f89c69a4-fba4-41a7-b8e2-f0506fa2cf2e,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Sangat 

In [131]:
# PELABELAN
import csv
import requests
from io import StringIO

# membaca data kamus kata-kata positif dari github
lexicon_positive = dict()

# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
if response.status_code == 200:
  # jika permintaan berhasil
  # membaca teks response sebagai file csv menggunakan pembaca csv dengan pemisah koma
  reader = csv.reader(StringIO(response.text), delimiter=',')

  for row in reader:
    # mengulangi setiap baris dalam file csv
    lexicon_positive[row[0]] = int(row[1])

    # menambahkan kata-kata positif dan skornya ke dalam kamus lexicon_positive
else:
  print('Failed to fetch positive lexicon data')




# membaca data kamus kata-kata negatif dari github
lexicon_negative = dict()

# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_negative[row[0]] = int(row[1])
        # Menambahkan kata-kata negatif dan skornya dalam kamus lexicon_negative
else:
    print("Failed to fetch negative lexicon data")

In [132]:
# Fungsi untuk menentukan polaritas sentimen dari tweet

def sentiment_analysis_lexicon_indonesia(text):
    #for word in text:

    score = 0
    # Inisialisasi skor sentimen ke 0

    for word in text:
        # Mengulangi setiap kata dalam teks

        if (word in lexicon_positive):
            score = score + lexicon_positive[word]
            # Jika kata ada dalam kamus positif, tambahkan skornya ke skor sentimen

    for word in text:
        # Mengulangi setiap kata dalam teks (sekali lagi)

        if (word in lexicon_negative):
            score = score + lexicon_negative[word]
            # Jika kata ada dalam kamus negatif, kurangkan skornya dari skor sentimen

    polarity=''
    # Inisialisasi variabel polaritas

    if (score >= 0):
        polarity = 'positive'
        # Jika skor sentimen lebih besar atau sama dengan 0, maka polaritas adalah positif
    elif (score < 0):
        polarity = 'negative'
        # Jika skor sentimen kurang dari 0, maka polaritas adalah negatif

    # else:
    #     polarity = 'neutral'
    # Ini adalah bagian yang bisa digunakan untuk menentukan polaritas netral jika diperlukan

    return score, polarity
    # Mengembalikan skor sentimen dan polaritas teks

In [133]:
results = clean_df['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
clean_df['polarity_score'] = results[0]
clean_df['polarity'] = results[1]
print(clean_df['polarity'].value_counts())

polarity
negative    3564
positive    1338
Name: count, dtype: int64


In [136]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Pisahkan data menjadi fitur (tweet) dan label (sentimen)
X = clean_df['text_akhir']
y = clean_df['polarity']

# Ekstraksi fitur dengan TF-IDF
tfidf = TfidfVectorizer(max_features=200, min_df=17, max_df=0.8 )
X_tfidf = tfidf.fit_transform(X)

# Konversi hasil ekstraksi fitur menjadi dataframe
features_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())

# Menampilkan hasil ekstraksi fitur
features_df



,adil,akun,aneh,apk,aplikasi,away,bagus,banget,beli,bener,...,timnya,tolol,tolong,turun,udah,ulang,uninstall,update,war,wifi
0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.113666,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
1,0.159172,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.132978,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
2,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.346334,0.0,0.0
3,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.167789,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
4,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.087228,0.119075,0.0,0.309569,...,0.0,0.0,0.000000,0.0,0.221467,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4897,0.175403,0.000000,0.0,0.0,0.0,0.000000,0.107346,0.000000,0.0,0.000000,...,0.0,0.0,0.285894,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
4898,0.000000,0.000000,0.0,0.0,0.0,0.428253,0.264558,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
4899,0.000000,0.345196,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.220195,0.0,0.0
4900,0.000000,0.000000,0.0,0.0,0.0,0.181897,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.142649,0.0,0.0,0.000000,0.0,0.0


In [137]:
# Bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

In [138]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression


# Menentukan parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l2', 'l1'],
    'solver': ['liblinear', 'saga']
}

# Membuat objek Logistic Regression
logistic_regression = LogisticRegression()

# GridSearchCV untuk tuning hyperparameter
grid_search = GridSearchCV(estimator=logistic_regression, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Melatih model
grid_search.fit(X_train.toarray(), y_train)

# Menampilkan hyperparameter terbaik
print("Best Parameters: ", grid_search.best_params_)

# Prediksi dan evaluasi
best_lr = grid_search.best_estimator_
y_pred_test_lr = best_lr.predict(X_test.toarray())
accuracy_test_lr = accuracy_score(y_pred_test_lr, y_test)
print('Logistic Regression - accuracy_test after tuning:', accuracy_test_lr)


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best Parameters:  {'C': 1, 'penalty': 'l1', 'solver': 'saga'}
Logistic Regression - accuracy_test after tuning: 0.855249745158002
